In [95]:
from collections import deque

In [100]:
def tic_tac():
    print('Bim')
    yield
    print('Bam')
    yield from spam()
    return 'BOOOOM'

def spam():
    print('SPAM')
    yield
    print('OK SPAM')
    yield

In [101]:
co = tic_tac()

In [105]:
next(co)

StopIteration: BOOOOM

In [191]:
STATUS_NEW = 'NEW'
STATUS_RUNNING = 'RUNNING'
STATUS_FINISHED = 'FINISHED'
STATUS_ERROR = 'ERROR'
STATUS_CANCELLED = 'CANCELLED'

class Task:
    def __init__(self, coro):
        self.coro = coro
        self.name = coro.__name__
        self.status = STATUS_NEW
        self.return_value = None
        self.error_value = None
        
    def run(self):
        try:
            self.status = STATUS_RUNNING
            next(self.coro)
        except StopIteration as err:
            self.status = STATUS_FINISHED
            self.return_value = err.value
        except Exception as err:
            self.status = STATUS_ERROR
            self.error_value = err
            
    def cancel(self):
        if self.is_done():
            return
        self.status = STATUS_CANCELLED
            
    def is_done(self):
        return self.status in {STATUS_FINISHED, STATUS_ERROR}
    
    def is_cancelled(self):
        return self.status == STATUS_CANCELLED
    
    def __repr__(self):
        result = ''
        if self.is_done():
            result = '({!r})'.format(self.return_value or self.error_value)
        
        return '<Task "{}" [{}] {}>'.format(self.name, self.status, result)

In [153]:
task = Task(tic_tac())

In [154]:
while not task.is_done():
    task.run()
    print(task)

Bim
<Task "tic_tac" [RUNNING] >
Bam
SPAM
<Task "tic_tac" [RUNNING] >
OK SPAM
<Task "tic_tac" [RUNNING] >
<Task "tic_tac" [FINISHED] ('BOOOOM')>


In [155]:
running_tasks = deque()
running_tasks.append(Task(tic_tac()))
running_tasks.append(Task(tic_tac()))
running_tasks

deque([<Task "tic_tac" [NEW] >, <Task "tic_tac" [NEW] >])

In [156]:
while running_tasks:
    task = running_tasks.popleft()
    task.run()
    if not task.is_done():
        running_tasks.append(task)
    else:
        print(task)

Bim
Bim
Bam
SPAM
Bam
SPAM
OK SPAM
OK SPAM
<Task "tic_tac" [FINISHED] ('BOOOOM')>
<Task "tic_tac" [FINISHED] ('BOOOOM')>


In [158]:
class Loop:
    def __init__(self):
        self._running = deque()
        
    def schedule(self, task):
        if not isinstance(task, Task):
            task = Task(task)
        self._running.append(task)
        return task
    
    def run_until_empty(self):
        while self._running:
            self._loop()
            
    def run_until_complete(self, task):
        task = self.schedule(task)
        while not task.is_done():
            self._loop()
            
    def _loop(self):
        task = self._running.popleft()
        
        if task.is_cancelled():
            print(task)
            return
        
        task.run()
        if task.is_done():
            print(task)
            return
        
        self.schedule(task)

In [159]:
event_loop = Loop()
task1 = Task(tic_tac())
task2 = Task(tic_tac())

event_loop.schedule(task1)
event_loop.schedule(task2)

<Task "tic_tac" [NEW] >

In [160]:
event_loop.run_until_empty()

Bim
Bim
Bam
SPAM
Bam
SPAM
OK SPAM
OK SPAM
<Task "tic_tac" [FINISHED] ('BOOOOM')>
<Task "tic_tac" [FINISHED] ('BOOOOM')>


In [161]:
event_loop = Loop()
event_loop.run_until_complete(Task(tic_tac()), Task(tic_tac()))

TypeError: run_until_complete() takes 2 positional arguments but 3 were given

In [162]:
for _ in range(5):
    print(_)

0
1
2
3
4


In [166]:
def ensure_future(coro, loop=None):
    if not loop:
        loop = DEFAULT_LOOP
    return loop.schedule(coro)

In [205]:
DEFAULT_LOOP = Loop()

def cancel(task):
    task.cancel()
    yield

def sub_task():
    print('Task subtask')
    for _ in range(5):
        print('(substask)')
        yield

def example():
    print('Task example')
    print('Subtask task is starting ...')
    sub = ensure_future(sub_task())
    print('Return to example')
    for _ in range(3):
        print('(example)')
        yield
    yield from cancel(sub)
    
    

In [206]:
event_loop = DEFAULT_LOOP

In [207]:
event_loop.run_until_complete(example())

Task example
Subtask task is starting ...
Return to example
(example)
Task subtask
(substask)
(example)
(substask)
(example)
(substask)
<Task "sub_task" [CANCELLED] >
<Task "example" [FINISHED] (None)>


In [195]:
event_loop.run_until_empty()